In [1]:
import pandas as pd
import numpy as np
import plotly_express as px

## Data Importing


In [2]:
years = [year for year in range(2009,2020)]
# Collect the play-by-play data from pre/reg/post season
preseason_pbp = {}
regseason_pbp = {}
pstseason_pbp = {}

for year in years:
    preseason_pbp[year] = pd.read_csv(f"../data/pbp_data/pre_season/pre_pbp_{year}.csv", low_memory=False)
    regseason_pbp[year] = pd.read_csv(f"../data/pbp_data/regular_season/reg_pbp_{year}.csv", low_memory=False)
    pstseason_pbp[year] = pd.read_csv(f"../data/pbp_data/post_season/post_pbp_{year}.csv", low_memory=False)

In [3]:
all_preseason = pd.concat(preseason_pbp)
all_preseason['season_type'] = "pre"
all_regseason = pd.concat(regseason_pbp)
all_regseason['season_type'] = "reg"
all_pstseason = pd.concat(pstseason_pbp)
all_pstseason['season_type'] = "post"
all_pbp = pd.concat([all_preseason, all_regseason, all_pstseason])

initial_shape = all_pbp.shape
print(initial_shape)

(645303, 258)


## Data Cleaning

In [4]:
all_pbp.reset_index(names=['season','old_index'], inplace=True)
all_pbp.head()

,season,old_index,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,...,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season_type,Unnamed: 0
0,2009,0,37,2009080950,TEN,BUF,TEN,home,BUF,BUF,...,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,pre,NaN
1,2009,1,59,2009080950,TEN,BUF,TEN,home,BUF,TEN,...,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,pre,NaN
2,2009,2,83,2009080950,TEN,BUF,TEN,home,BUF,TEN,...,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,pre,NaN
3,2009,3,104,2009080950,TEN,BUF,TEN,home,BUF,TEN,...,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,pre,NaN
4,2009,4,128,2009080950,TEN,BUF,TEN,home,BUF,TEN,...,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,pre,NaN


In [5]:
# drop some columns we don't need
all_pbp.drop(['Unnamed: 0', 'old_index'], axis=1, inplace=True)

In [6]:
def nanalysis(df: pd.DataFrame, show=False):
    """
    formatted output of numeric features and the number of NaNs detected in
    the dataframe for each feature. Useful for deciding how to handle NaNs
    for different features. NaN analysis = NaNalysis
    """
    # get numeric columns as a subset
    df_numeric = df.select_dtypes(include='number')
    # let's find the NaN values in the numeric columns
    output = {}
    features_left = 0   # track features remaining with NaNs
    if show:
        print("Feature   --->   NaNs")
    for f in df_numeric.columns.tolist():
        nan_count = np.sum(np.isnan(df[f]))
        if show and nan_count > 0:
            print(f"{f}   --->   {nan_count}")
            features_left += 1
        output[f] = nan_count
    if not features_left:
        print("No NaNs remaining in numeric data! Well Done!")
    return output

In [7]:
nan_results = nanalysis(all_pbp, show=True)

Feature   --->   NaNs
yardline_100   --->   19473
quarter_seconds_remaining   --->   205
half_seconds_remaining   --->   205
game_seconds_remaining   --->   205
down   --->   96510
goal_to_go   --->   18967
yards_gained   --->   213
qb_dropback   --->   19195
air_yards   --->   394125
yards_after_catch   --->   490292
kick_distance   --->   573057
timeout   --->   19195
posteam_timeouts_remaining   --->   19218
defteam_timeouts_remaining   --->   19218
posteam_score   --->   22875
defteam_score   --->   22875
score_differential   --->   22875
posteam_score_post   --->   19218
defteam_score_post   --->   19218
score_differential_post   --->   19218
no_score_prob   --->   1084
opp_fg_prob   --->   1084
opp_safety_prob   --->   1084
opp_td_prob   --->   1084
fg_prob   --->   1084
safety_prob   --->   1084
td_prob   --->   1084
ep   --->   20288
epa   --->   23538
air_epa   --->   396364
yac_epa   --->   396864
comp_air_epa   --->   19475
comp_yac_epa   --->   19693
wp   --->   23882
def_w

In [8]:
# For some reason, these two rows are read in as numeric data, though they shouldn't be
# let's drop them. They are mostly NaNs anyway
all_pbp = all_pbp.drop(columns=['lateral_sack_player_id','lateral_sack_player_name'])

In [9]:
# Let's start with the rows with the most NaNs
big_NAN = ['air_yards', 'yards_after_catch', 'kick_distance', 
           'air_epa', 'yac_epa', 'air_wpa', 'yac_wpa', 'penalty_yards',
           'fumble_recovery_1_yards', 'fumble_recovery_2_yards']
for feature in big_NAN:
    print(feature, " --> ", nan_results[feature])

air_yards  -->  394125
yards_after_catch  -->  490292
kick_distance  -->  573057
air_epa  -->  396364
yac_epa  -->  396864
air_wpa  -->  396559
yac_wpa  -->  396858
penalty_yards  -->  596996
fumble_recovery_1_yards  -->  636444
fumble_recovery_2_yards  -->  645233


In [10]:
# For the sake of our machine learning model, we can probably do this
for feature in big_NAN:
    all_pbp[feature] = all_pbp[feature].fillna(value=0)

In [11]:
nan_results = nanalysis(all_pbp, show=True)

Feature   --->   NaNs
yardline_100   --->   19473
quarter_seconds_remaining   --->   205
half_seconds_remaining   --->   205
game_seconds_remaining   --->   205
down   --->   96510
goal_to_go   --->   18967
yards_gained   --->   213
qb_dropback   --->   19195
timeout   --->   19195
posteam_timeouts_remaining   --->   19218
defteam_timeouts_remaining   --->   19218
posteam_score   --->   22875
defteam_score   --->   22875
score_differential   --->   22875
posteam_score_post   --->   19218
defteam_score_post   --->   19218
score_differential_post   --->   19218
no_score_prob   --->   1084
opp_fg_prob   --->   1084
opp_safety_prob   --->   1084
opp_td_prob   --->   1084
fg_prob   --->   1084
safety_prob   --->   1084
td_prob   --->   1084
ep   --->   20288
epa   --->   23538
comp_air_epa   --->   19475
comp_yac_epa   --->   19693
wp   --->   23882
def_wp   --->   23882
home_wp   --->   20796
away_wp   --->   20796
wpa   --->   7669
home_wp_post   --->   23666
away_wp_post   --->   23666
c

In [12]:
# Let's look at what's going on with NaNs in the "yardline_100" column
count = 0
for row in all_pbp.itertuples():
    if np.isnan(row.yardline_100):
        count += 1
        print(row.desc)
    if count > 25:
        break

Timeout #1 by TEN at 01:12.
Timeout #1 by BUF at 00:17.
Timeout #2 by BUF at 00:10.
Timeout #1 by BUF at 07:17.
END QUARTER 3
Two-Minute Warning
END QUARTER 3
(13:12) (Shotgun) A.Walter pass incomplete to S.Morris (J.Babin). Penalty on PHI-M.Fokou, Illegal Use of Hands, offsetting, enforced at PHI 35 - No Play.
Two-Minute Warning
END GAME
Timeout #1 by BAL at 07:48.
Timeout #1 by WAS at 06:23.
Timeout #2 by BAL at 11:27.
Timeout #3 by BAL at 00:59.
Timeout #1 by WAS at 13:31.
Two-Minute Warning
Timeout #1 by ARI at 13:24.
Timeout #2 by ARI at 02:42.
Timeout #3 by ARI at 00:53.
Timeout #1 by PIT at 10:20.
Two-Minute Warning
END GAME
Timeout #1 by DAL at 12:26.
END QUARTER 1
Two-Minute Warning
END QUARTER 2


In [13]:
# They seem to be almost all Non-play rows, so we can probably safely drop them
all_pbp = all_pbp.dropna(subset='yardline_100')

In [14]:
nan_results = nanalysis(all_pbp, show=True)

Feature   --->   NaNs
quarter_seconds_remaining   --->   6
half_seconds_remaining   --->   6
game_seconds_remaining   --->   6
down   --->   77786
yards_gained   --->   213
qb_dropback   --->   251
timeout   --->   251
posteam_timeouts_remaining   --->   251
defteam_timeouts_remaining   --->   251
posteam_score   --->   3908
defteam_score   --->   3908
score_differential   --->   3908
posteam_score_post   --->   251
defteam_score_post   --->   251
score_differential_post   --->   251
no_score_prob   --->   1084
opp_fg_prob   --->   1084
opp_safety_prob   --->   1084
opp_td_prob   --->   1084
fg_prob   --->   1084
safety_prob   --->   1084
td_prob   --->   1084
ep   --->   1343
epa   --->   4594
comp_air_epa   --->   531
comp_yac_epa   --->   749
wp   --->   4970
def_wp   --->   4970
home_wp   --->   4920
away_wp   --->   4920
wpa   --->   7580
home_wp_post   --->   7790
away_wp_post   --->   7790
comp_air_wpa   --->   726
comp_yac_wpa   --->   853
punt_blocked   --->   251
first_down_r

In [15]:
# What about all the columns with 248 NaNs left?
count = 0
for row in all_pbp.itertuples():
    if np.isnan(row.timeout):
        count += 1
        print(row.desc)
    if count > 25:
        break

END QUARTER 3
Two-Minute Warning
END QUARTER 3
END QUARTER 3
Two-Minute Warning
END GAME
Two-Minute Warning
END QUARTER 1
END QUARTER 3
END GAME
Two-Minute Warning
END QUARTER 1
END GAME
END QUARTER 1
Two-Minute Warning
Two-Minute Warning
END GAME
END GAME
Two-Minute Warning
END GAME
END GAME
The game has been suspended. Clock time was 742pm
Two-Minute Warning
Two-Minute Warning
END GAME
END QUARTER 3


In [16]:
# They are all NONPLAYS, so remove these rows
all_pbp = all_pbp.dropna(subset='timeout')

In [17]:
nan_results = nanalysis(all_pbp, show=True)

Feature   --->   NaNs
quarter_seconds_remaining   --->   3
half_seconds_remaining   --->   3
game_seconds_remaining   --->   3
down   --->   77545
yards_gained   --->   213
posteam_score   --->   3657
defteam_score   --->   3657
score_differential   --->   3657
no_score_prob   --->   1084
opp_fg_prob   --->   1084
opp_safety_prob   --->   1084
opp_td_prob   --->   1084
fg_prob   --->   1084
safety_prob   --->   1084
td_prob   --->   1084
ep   --->   1092
epa   --->   4343
comp_air_epa   --->   280
comp_yac_epa   --->   498
wp   --->   4720
def_wp   --->   4720
home_wp   --->   4720
away_wp   --->   4720
wpa   --->   7579
home_wp_post   --->   7590
away_wp_post   --->   7590
comp_air_wpa   --->   475
comp_yac_wpa   --->   602
return_yards   --->   21


In [18]:
# Okay, what's going on with NaNs in 'down'?
count = 0
for row in all_pbp.itertuples():
    if np.isnan(row.down):
        count += 1
        print(row.desc)
    if count > 25:
        break

R.Lindell kicks 68 yards from BUF 30 to TEN 2. R.Mouton to TEN 26 for 24 yards (D.Fine).
R.Bironas extra point is GOOD, Center-K.Amato, Holder-A.Trapasso.
R.Bironas kicks 66 yards from TEN 30 to BUF 4. B.Hall to BUF 23 for 19 yards (C.Cramer).
R.Bironas extra point is GOOD, Center-K.Amato, Holder-A.Trapasso.
R.Bironas kicks 70 yards from TEN 30 to BUF 0. B.Hall to BUF 18 for 18 yards (C.Allred). PENALTY on BUF-D.Fine, Illegal Block Above the Waist, 9 yards, enforced at BUF 18.
R.Lindell kicks 66 yards from BUF 30 to TEN 4. R.Mouton to TEN 28 for 24 yards (J.Wendling). PENALTY on TEN-M.Otto, Offensive Holding, 10 yards, enforced at TEN 22.
R.Bironas extra point is GOOD, Center-K.Amato, Holder-A.Trapasso.
A.Trapasso kicks 63 yards from TEN 30 to BUF 7. B.Hall to BUF 37 for 30 yards (A.Trapasso).
A.Trapasso kicks 64 yards from TEN 30 to BUF 6. C.Hawthorne to BUF 24 for 18 yards (J.Ringer).
R.Lindell kicks 64 yards from BUF 30 to TEN 6. J.McCourty to TEN 14 for 8 yards (A.Palmer).
R.Lindel

In [19]:
# They're all kicks. Let's fill these as "zeroth down"
all_pbp.loc[:,'down'] = all_pbp['down'].fillna(value=0)

In [20]:
nan_results = nanalysis(all_pbp, show=True)

Feature   --->   NaNs
quarter_seconds_remaining   --->   3
half_seconds_remaining   --->   3
game_seconds_remaining   --->   3
yards_gained   --->   213
posteam_score   --->   3657
defteam_score   --->   3657
score_differential   --->   3657
no_score_prob   --->   1084
opp_fg_prob   --->   1084
opp_safety_prob   --->   1084
opp_td_prob   --->   1084
fg_prob   --->   1084
safety_prob   --->   1084
td_prob   --->   1084
ep   --->   1092
epa   --->   4343
comp_air_epa   --->   280
comp_yac_epa   --->   498
wp   --->   4720
def_wp   --->   4720
home_wp   --->   4720
away_wp   --->   4720
wpa   --->   7579
home_wp_post   --->   7590
away_wp_post   --->   7590
comp_air_wpa   --->   475
comp_yac_wpa   --->   602
return_yards   --->   21


In [21]:
# For the remaining NaNs, the best bet is probably just to drop everything
# This shouldn't hurt our data too much
pre_length = len(all_pbp)
all_pbp = all_pbp.dropna(subset=nan_results.keys())
print('Rows Dropped:', len(all_pbp) - pre_length)
print('Remaining Plays:', len(all_pbp))

Rows Dropped: -11082
Remaining Plays: 614497


In [22]:
final_shape = all_pbp.shape
print(initial_shape, " --> ", final_shape)

(645303, 258)  -->  (614497, 256)


In [23]:
nan_results = nanalysis(all_pbp, show=True)

Feature   --->   NaNs
No NaNs remaining in numeric data! Well Done!


In [24]:
# Send back to csv files
for year in years:
    print("Saving data for", year)
    # filter df for each year
    year_df = all_pbp[all_pbp['season'] == year]
    # filter each year df for pre/reg/post
    pre_df = year_df[year_df['season_type'] == "pre"]
    pst_df = year_df[year_df['season_type'] == "post"]
    reg_df = year_df[year_df['season_type'] == "reg"]
    # send to files
    pre_df.to_csv(f"../cleaned_data/pbp_data/pre_season/pre_pbp_{year}.csv")
    pst_df.to_csv(f"../cleaned_data/pbp_data/post_season/post_pbp_{year}.csv")
    reg_df.to_csv(f"../cleaned_data/pbp_data/regular_season/reg_pbp_{year}.csv")

Saving data for 2009
Saving data for 2010
Saving data for 2011
Saving data for 2012
Saving data for 2013
Saving data for 2014
Saving data for 2015
Saving data for 2016
Saving data for 2017
Saving data for 2018
Saving data for 2019
